Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 2: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [2]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [3]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [4]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [5]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [6]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [7]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [8]:
import numpy as np

def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


# My method
this method provides to create random moves and select the best move. The best move is the one that has the fewest bits at 1 when it executes the nim_sum. If the nim_sum is 0, it will select that, instead if it is not 0, it will find out the best move near to this one.

In [18]:
def PosNormal(mean, sigma, num_obj):
    x = np.random.normal(mean,sigma)
    return(x if x>=1 and x<num_obj else PosNormal(mean, sigma, num_obj))

def count_ones(num):
    count = 0
    while num:
        num &= (num - 1)
        count += 1
    return count

def tweak(state: Nim, move: Nimply) -> Nimply:               #it takes in input Nim and Nimply and search a move near to its
    to_remove = int(PosNormal(move.num_objects, (state.rows[move.row]/2), (state.rows[move.row]+1)))#PosNormal(mean, sigma, number of objs)
    return Nimply(move.row, to_remove)

def my_strategy(state: Nim) -> Nimply:
    non_empty_row = [r for r, c in enumerate(state.rows) if c != 0]
    #new_state = tweak(state, non_empty_row[random.randint(0,len(non_empty_row)-1)])
    new_state = []          #array di nuove mosse
    if (len(non_empty_row) == 1):
        if (state.rows[non_empty_row[0]] == 1):
            return Nimply(non_empty_row[0], 1)
        return Nimply(non_empty_row[0], state.rows[non_empty_row[0]]-1)
    for _ in range(int(sum(state.rows[j] for j in non_empty_row)/3)+1):
        tmp = deepcopy(state)
        r_var = pure_random(state)
        tmp.nimming(r_var)
        new_state.append([r_var, nim_sum(tmp)])
    new_state.sort(key=lambda x: count_ones(x[1]))
    print("----------------------------------")
    print("original",new_state)
    if new_state[0][1] == 0:
        return new_state[0][0]              #best solution early found
    mutated = [new_state[0]]
    for _ in range(int(state.rows[new_state[0][0].row]/2)):
        tmp = deepcopy(state)
        r_var = tweak(state,new_state[0][0])
        tmp.nimming(r_var)
        mutated.append([r_var, nim_sum(tmp)])
    mutated.sort(key=lambda x: count_ones(x[1]))
    print("mutated",mutated)
    return mutated[0][0]

## Oversimplified match

In [19]:
logging.getLogger().setLevel(logging.INFO)

strategy = (my_strategy, pure_random)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <1 3 5 6 9>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=1)
INFO:root:status: <1 3 5 5 9>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=3)
INFO:root:status: <1 0 5 5 9>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 0 5 5 9>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 0 4 5 9>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=5)
INFO:root:status: <0 0 4 5 4>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=3)
INFO:root:status: <0 0 4 5 1>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 0 4 4 1>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1)
INFO:root:status: <0 0 4 4 0>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=3)
INFO:root:status: <0 0 4 1 0>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 0

----------------------------------
original [[Nimply(row=3, num_objects=1), 8], [Nimply(row=0, num_objects=1), 8], [Nimply(row=2, num_objects=1), 8], [Nimply(row=0, num_objects=1), 8], [Nimply(row=2, num_objects=1), 8], [Nimply(row=2, num_objects=5), 12], [Nimply(row=4, num_objects=2), 7], [Nimply(row=4, num_objects=2), 7], [Nimply(row=4, num_objects=2), 7]]
mutated [[Nimply(row=3, num_objects=1), 8], [Nimply(row=3, num_objects=1), 8], [Nimply(row=3, num_objects=1), 8], [Nimply(row=3, num_objects=4), 13]]
----------------------------------
original [[Nimply(row=1, num_objects=1), 10], [Nimply(row=3, num_objects=1), 10], [Nimply(row=4, num_objects=1), 10], [Nimply(row=0, num_objects=1), 10], [Nimply(row=0, num_objects=1), 10], [Nimply(row=4, num_objects=8), 3], [Nimply(row=2, num_objects=3), 12], [Nimply(row=1, num_objects=2), 9]]
mutated [[Nimply(row=1, num_objects=3), 8], [Nimply(row=1, num_objects=1), 10]]
----------------------------------
original [[Nimply(row=2, num_objects=1), 8]